In [1]:
from IJCAI2017_TOOL import *
import xgboost as xgb

In [2]:
X = pd.read_csv('./Flamingo_Rank4_code/feature/X.csv')
Y = pd.read_csv('./Flamingo_Rank4_code/feature/Y.csv')
Xtest = pd.read_csv('./Flamingo_Rank4_code/feature/Xtest.csv',low_memory=False)
fix = pd.read_csv('./Flamingo_Rank4_code/feature/FIXER.csv')

In [16]:
X.head()

,SA00,SA01,SA02,SA03,SA04,SA05,SA06,SA07,SA08,SA09,...,RJ04,RJ05,RJ06,RJ07,RJ08,RJ09,RJ10,RJ11,RJ12,RJ13
0,0.894646,0.914277,0.853042,0.802678,0.851797,0.937811,1.027635,0.872442,0.916253,0.804705,...,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000
1,0.827307,0.847390,0.737592,0.740289,0.784993,0.737592,0.793118,0.809389,0.900038,0.742947,...,1.0,1.000000,0.894737,1.0,1.000000,0.947368,1.0,1.0,1.000000,0.684211
2,0.748668,0.617894,0.617894,0.636357,0.549606,0.636357,0.607992,0.644995,0.867961,0.820549,...,1.0,1.000000,0.894737,1.0,1.000000,0.947368,1.0,1.0,1.000000,0.684211
3,0.260365,0.260365,0.424637,0.260365,0.164272,0.260365,0.328544,0.328544,0.328544,0.607879,...,1.0,0.894737,1.000000,1.0,0.947368,1.000000,1.0,1.0,0.684211,0.789474
4,0.309597,0.266672,0.266672,0.266672,0.400008,0.309597,0.461267,0.545006,0.344669,0.374322,...,1.0,1.000000,0.812500,1.0,1.000000,1.000000,1.0,1.0,0.875000,1.000000


In [17]:
Y.head()

,SB00,SB01,SB02,SB03,SB04,SB05,SB06,SB07,SB08,SB09,SB10,SB11,SB12,SB13
0,0.943032,0.913451,0.869976,0.806131,0.869976,0.854498,0.874878,0.646839,0.700098,0.681759,0.694171,0.594751,0.737266,0.784451
1,1.044207,1.026267,0.900038,0.873162,0.900038,0.927121,0.919520,1.004960,1.006646,0.909535,0.905377,0.896756,0.905377,0.952220
2,1.055050,1.032021,0.983141,0.928450,0.913249,0.922074,1.002318,1.024342,1.032542,0.962997,0.953976,0.961225,0.946458,0.994030
3,1.061104,0.992925,0.978108,0.880096,0.966354,1.020040,0.912464,0.974254,0.978108,0.940937,0.992925,1.050079,1.096093,1.109639
4,0.763738,0.666680,0.744677,0.723517,0.709605,0.709605,0.718990,0.752530,0.760074,0.777735,0.748643,0.770864,0.796987,0.873112


In [18]:
fix.head()

,SHOP_ID,VALUE
0,1,255.0
1,2,83.0
2,3,73.0
3,4,107.0
4,5,204.0


In [3]:
TRN_N = 21
TST_N = 14
TST_PAD_N = 14 + 4

TRN_RANGE = np.arange(0,TRN_N)
TST_RANGE = np.arange(0,TST_N)

TRAIN_TRN_C = [(lambda x:('SA'+ str(x).zfill(2))) (x)  for x in TRN_RANGE]
TRAIN_TST_C = [(lambda x:('SB'+ str(x).zfill(2))) (x)  for x in TST_RANGE] 

SHOP_SC = ['SC00']
SHOP_SD = [(lambda x:('SD'+ str(x).zfill(2))) (x)  for x in range(5)]
SHOP_SE =  [(lambda x:('SE'+ str(x).zfill(2))) (x)  for x in range(1)]
SHOP_SF = [(lambda x:('SF'+ str(x).zfill(2))) (x)  for x in range(1)]
SHOP_SG = [(lambda x:('SG'+ str(x).zfill(2))) (x)  for x in range(4)]
SHOP_SH = [(lambda x:('SH'+ str(x).zfill(2))) (x)  for x in range(1,2)]
SHOP_SI = [(lambda x:('SI'+ str(x).zfill(2))) (x)  for x in range(10)]
SHOP_SJ = [(lambda x:('SJ'+ str(x).zfill(2))) (x)  for x in range(15)]
SHOP_columns = SHOP_SC +SHOP_SD+SHOP_SE+SHOP_SF+SHOP_SG + SHOP_SH  + SHOP_SI + SHOP_SJ

WEARC_TRN_C = [(lambda x:('RC'+ str(x).zfill(2))) (x)  for x in range(TRN_N)]
WEARD_TST_C = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(TST_N)]
WEARE_TRN_C = [(lambda x:('RE'+ str(x).zfill(2))) (x)  for x in range(TRN_N)]
WEARF_TST_C = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(TST_N)]
WEARG_TRN_C = [(lambda x:('RG'+ str(x).zfill(2))) (x)  for x in range(TRN_N)]
WEARH_TST_C = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(TST_N)]
WEARI_TRN_C = [(lambda x:('RI'+ str(x).zfill(2))) (x)  for x in range(TRN_N)]
WEARJ_TST_C = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(TST_N)]
weather_output_columns = WEARC_TRN_C + WEARD_TST_C + WEARE_TRN_C + WEARF_TST_C +WEARG_TRN_C + WEARH_TST_C  + WEARI_TRN_C + WEARJ_TST_C 

HOLI_TRN_CA  = [(lambda x:('NC'+ str(x).zfill(2))) (x)  for x in TRN_RANGE]
HOLI_TST_CA  = [(lambda x:('ND'+ str(x).zfill(2))) (x)  for x in range(-2,TST_PAD_N-2)]
HOLI_RATIO = [(lambda x:('NE'+ str(x).zfill(2))) (x)  for x in range(1)]

PRECIP_TRN_C = [(lambda x:('RA'+ str(x).zfill(2))) (x)  for x in TRN_RANGE]
PRECIP_TST_C = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(TST_N)]

In [4]:
#处理销量特征

#用平均数填空值
X[TRAIN_TRN_C] = X[TRAIN_TRN_C].T.fillna(X[TRAIN_TRN_C].mean(axis = 1)).T
Xtest[TRAIN_TRN_C] = Xtest[TRAIN_TRN_C].T.fillna(Xtest[TRAIN_TRN_C].mean(axis = 1)).T
#取中位数
TRN_BASE = X[TRAIN_TRN_C].median(axis=1)
TST_BASE = Xtest[TRAIN_TRN_C].median(axis = 1)
#减去中位数
X[TRAIN_TRN_C] = X[TRAIN_TRN_C].sub(TRN_BASE , axis =0)
Xtest[TRAIN_TRN_C] = Xtest[TRAIN_TRN_C].sub(TST_BASE , axis =0)
Y[TRAIN_TST_C] = Y[TRAIN_TST_C].sub(TRN_BASE , axis =0)

In [5]:
def abs_relative_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)/np.abs(y_pred+y_true)) )
def abs_relative_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_pred+y_true)
    
def abs_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)))
    
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true) 

In [6]:
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500) 

In [8]:
COLUMN_ALL = TRAIN_TRN_C + HOLI_TRN_CA +  HOLI_TST_CA  + HOLI_RATIO + SHOP_columns + PRECIP_TRN_C  + WEARC_TRN_C  + WEARE_TRN_C + WEARG_TRN_C + WEARI_TRN_C 
Ytrain_all = pd.DataFrame()
error_list = []

In [9]:
for ind in [0,1]:#当天附近四天 （之前两天 当天 之后一天）
    if ind<= 1:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
    elif(ind<= 12):
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
    else:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )] 
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
    
    COLUMN_IND = COLUMN_IND1 +  COLUMN_IND2 +COLUMN_IND3 +  COLUMN_IND4  + COLUMN_IND5

    Xtrain_in = X[COLUMN_ALL + COLUMN_IND]
    
    time1 = time.time()
    XGBR.fit(Xtrain_in.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(Xtrain_in.values)
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    
    error_list.append(abs_error(y_pred, y_true  ))
    
    Ytrain =  XGBR.predict(Xtrain_in.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)

0_error:0.036863697083__time:852.578385115
1_error:0.0402167137008__time:871.136412144


In [10]:
for ind in [2,3,4,5,6,7,8,9,10,11,12,13]:#当天附近四天 （之前两天 当天 之后一天）
    if ind<= 1:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
    elif(ind<= 12):
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
    else:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND4 = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )] 
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
    
    COLUMN_IND = COLUMN_IND1 +  COLUMN_IND2 +COLUMN_IND3 +  COLUMN_IND4  + COLUMN_IND5

    Xtrain_in = X[COLUMN_ALL + COLUMN_IND]
    
    time1 = time.time()
    XGBR.fit(Xtrain_in.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(Xtrain_in.values)
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    
    error_list.append(abs_error(y_pred, y_true  ))
    
    Ytrain =  XGBR.predict(Xtrain_in.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)

2_error:0.0420962379141__time:884.366980076
3_error:0.0432321963134__time:879.466460943
4_error:0.0443384608115__time:920.336216211
5_error:0.0450985749534__time:877.515882015
6_error:0.0458420146818__time:910.630874157
7_error:0.0476827793458__time:880.672631025
8_error:0.0488253505449__time:883.27086401
9_error:0.0496376629678__time:880.324000835
10_error:0.0501276849647__time:880.299371958
11_error:0.0505944511042__time:881.078063011
12_error:0.0512676610238__time:879.559469938
13_error:0.0519604978228__time:860.792695045


In [12]:
Ytrain_all.head()

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.035277,0.037539,-0.014040,-0.019072,0.005982,0.017991,0.049336,0.033607,0.015422,-0.016958,-0.028206,-0.003123,0.017995,0.052348
1,0.189887,0.182572,0.092593,0.087968,0.074827,0.081759,0.110067,0.174975,0.167412,0.073663,0.070543,0.061470,0.049600,0.075014
2,0.191099,0.198574,0.128396,0.104918,0.114486,0.109035,0.117797,0.185937,0.181572,0.107624,0.078183,0.083579,0.069508,0.075573
3,0.508017,0.499269,0.482499,0.399506,0.421912,0.432883,0.412898,0.428542,0.362725,0.334588,0.400170,0.330419,0.334834,0.333656
4,0.269531,0.195294,0.216784,0.235082,0.225739,0.265522,0.245673,0.262127,0.227204,0.199142,0.221554,0.212927,0.238923,0.301205


In [13]:
print(np.mean(error_list))

0.0462702845166


In [14]:
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]#选出错误数较少的前90%的样本

In [17]:
Good_ind.size

422617

In [18]:
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.03,n_estimators=1600,reg_alpha=1,reg_lambda=0)
Ytest_all = pd.DataFrame()
error_list = []

In [19]:
for ind in [0,1,2,3,4,5,6,7,8,9,10,11,12,13]:
    
    if ind<= 1:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND4  = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(0 ,ind+2 )]
    elif(ind<= 12):
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND4  = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+2 )]

    else:
        COLUMN_IND1 = [(lambda x:('RB'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND2 = [(lambda x:('RF'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND3 = [(lambda x:('RH'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]
        COLUMN_IND4  = [(lambda x:('RJ'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )] 
        COLUMN_IND5 = [(lambda x:('RD'+ str(x).zfill(2))) (x)  for x in range(ind-2 ,ind+1 )]


    COLUMN_IND = COLUMN_IND1 +  COLUMN_IND2 +COLUMN_IND3 +  COLUMN_IND4  + COLUMN_IND5
    Xtrain_in = X[COLUMN_ALL + COLUMN_IND]
    Xtest_in = Xtest[COLUMN_ALL + COLUMN_IND]
    time1 = time.time()
    XGBR.fit(Xtrain_in.values[Good_ind],Y.values[Good_ind,ind] )
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(Xtrain_in.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest_in.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)   

0_error:0.0298940621942__time:4831.14603901
1_error:0.0313187617683__time:4927.11745501
2_error:0.032036066377__time:5056.9452219
3_error:0.0325481120981__time:5033.55736804
4_error:0.0328846901196__time:5015.1905942
5_error:0.0332177447378__time:5023.83408594
6_error:0.0335317434464__time:5029.65833712
7_error:0.0344282385823__time:5048.39637899
8_error:0.0349135697852__time:5038.48611212
9_error:0.0353470164768__time:5022.93186903
10_error:0.0357662065097__time:5037.13352609
11_error:0.0362629478518__time:5165.028337
12_error:0.0369430889941__time:5000.72037196
13_error:0.0378904079706__time:4939.85804892


In [20]:
print(np.mean(error_list))

0.0340701897794


In [21]:
result = Ytest_all.copy()
result.columns = [np.arange(14)]
result['SHOP_ID'] = np.arange(1,2001)
result = pd.merge(result ,fix,on='SHOP_ID',how = 'left')
result[np.arange(0,14)] = np.e**(result[np.arange(0,14)].sub(-TST_BASE, axis = 0).multiply(np.log(result['VALUE']),axis=0) )

In [22]:
result_sub = pd.concat((result['SHOP_ID'],result[np.arange(0,14)] ),axis = 1 )
result_sub[result_sub<0] = 5

In [23]:
filename = 'xgb_model1.csv'
result_sub.to_csv(filename,header=None, index = False)